In [1]:
#import dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 81.0.4044
Get LATEST driver version for 81.0.4044
Driver [/Users/nicolebroussard/.wdm/drivers/chromedriver/mac64/81.0.4044.138/chromedriver] found in cache


NASA Mars News

In [3]:
#Scrape Mars news site 
url = 'https://mars.nasa.gov/news/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
#print(soup.prettify())

In [4]:
#Retrieve the latest News Title and paragraph
news_title = soup.find('div', class_='content_title').find('a').text
news_p = soup.find('div', class_='rollover_description_inner').text
print(news_title)
print(news_p)


NASA's Ingenuity Mars Helicopter Completes First One-Way Trip  


The Red Planet rotorcraft headed south in support of furthering research into the potential use of aerial scouts on Mars in the future.



JPL Mars Space Images - Featured Image

In [5]:
#Scrape Mars Space Images
jpl_url = 'https://spaceimages-mars.com'
browser.visit(jpl_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify())

In [6]:
#Use splinter to navigate to image
mars_img = soup.find('img', class_='headerimage fade-in').get('src')
mars_img
featured_img_url = jpl_url+'/'+mars_img
featured_img_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

Mars Facts

In [7]:
#Scrape Mars Fact table
facts_url = 'https://galaxyfacts-mars.com'
mars_facts_table = pd.read_html(facts_url)
mars_facts_table

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [8]:
#Turn data into html
fact_df = mars_facts_table[0]
fact_df.columns = ['Mars-Earth Comparison', 'Mars', 'Earth']
html_table = fact_df.to_html()
fact_html_table = html_table.replace('\n', '')
fact_html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars-Earth Comparison</th>      <th>Mars</th>      <th>Earth</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Mars - Earth Comparison</td>      <td>Mars</td>      <td>Earth</td>    </tr>    <tr>      <th>1</th>      <td>Diameter:</td>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2</td>      <td>1</td>    </tr>    <tr>      <th>4</th>      <td>Distance from Sun:</td>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <th>5</th>      <td>Length of Year:</td>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <th>6</th>      <td>Temperature:</td>      <td>-87 to -5 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>'

Mars Hemispheres

In [9]:
#Obtain high resolution images for each of Mar's hemispheres
general_url = 'https://marshemispheres.com'
browser.visit(general_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify())

In [10]:
#Find href for each hemisphere
items = soup.find_all('div', class_='item')
urls = []

for item in items:
    url = item.find('a', class_='itemLink product-item')['href']
    urls.append(general_url+'/'+url)
print(urls)

['https://marshemispheres.com/cerberus.html', 'https://marshemispheres.com/schiaparelli.html', 'https://marshemispheres.com/syrtis.html', 'https://marshemispheres.com/valles.html']


In [11]:
titles = []
img_urls = []
hemisphere_image_urls = []

for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    item_url = soup.find('img', class_='wide-image').get('src')
    img_url = general_url+'/'+item_url
    img_urls.append(img_url)

    title = soup.find('h2', class_='title').text
    titles.append(title)

for i in range(len(titles)):
    hemisphere_image_url = {
            'title': titles[i],
            'img_url': img_urls[i]
        } 
    hemisphere_image_urls.append(hemisphere_image_url)
    browser.back()
    
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]
